This is the playground to test coneff code. The main implementation of coneff is in coneff.py

In [1]:
import numpy as np
import nibabel as nib

In [2]:
N = 10000
struc_mat = np.random.rand(N,N)
func_mat = np.random.rand(N,N)

In [3]:
def preprocess_BOLD_coneff(func_mat):
    #TODO: implement BOLD preprocessing
    return func_mat

In [ ]:
def threshold_struc_mat(struc_mat, threshold):
    return struc_mat * (struc_mat > threshold)

In [ ]:
def compute_2ndstep_mat(struc_mat):
    #TODO: implement 1 degree separation matrix
    return struc_mat

In [ ]:
def get_binary_mat(mat):
    return mat > 0

In [ ]:
def normlaize_mat(mat):
    max_val = np.max(mat)
    mat = mat / max_val
    return mat

In [ ]:
def gradient_descrent(error_th, maxiter, brain_par, skip_bold, averaged_bold_seq, M):
    iter = 0
    etem = []
    while error_des > error_th and iter < maxiter:
        error_des = 0
        gradient = np.zeros((brain_par, brain_par))

        for jj in range(skip_bold, t - 1):
            error_des += np.linalg.norm(
                np.dot(M, averaged_bold_seq[jj, :]) - averaged_bold_seq[jj + 1, :]
            )

        error_des *= 0.5
        etem.append(error_des)

        for jj in range(skip_bold, t - 1):
            if use_multistep:
                gradient += (
                    np.dot(M, averaged_bold_seq[jj, :])
                    + np.dot(M_2nstep, averaged_bold_seq[jj, :])
                    - averaged_bold_seq[jj + 1, :]
                )
            else:
                gradient += (
                    np.dot(M, averaged_bold_seq[jj, :]) - averaged_bold_seq[jj + 1, :]
                )

        M = M - eta * gradient
        M = M * noconn_M
        iter += 1

In [1]:
def structured_G_causality(struc_mat, bold_seq, norm_opt, use_multistep, use_deconv):
    #TODO: check data structure of struc_mat and bold_seq
    
    brain_par = 264
    maxiter = 5000
    maxiter2 = 5000
    error_th = 0.016
    eta = 0.000005
    eta2 = 0.00001
    skip_bold = 5
    structural_thres = 0.5

    averaged_bold_seq = preprocess_BOLD_coneff(bold_seq)
     
    structure_direct = threshold_struc_mat(struc_mat, structural_thres)
    structure_2nstep = threshold_struc_mat(compute_2ndstep_mat(struc_mat), structural_thres)

    if norm_opt == 1:
        print("Initial matrix binarized")
        M = get_binary_mat(structure_direct)
        M_2nstep = get_binary_mat(structure_2nstep)
        
    elif norm_opt == 2:
        print("Initial matrix normalized according to its largest value")
        M = normlaize_mat(structure_direct)
        M_2nstep = normlaize_mat(structure_2nstep)
        
    noconn_M = M.astype(bool)
    noconn_M2 = M_2nstep.astype(bool)

    '''
    gr
    '''
    

    if use_multistep:
        error_des = 1e10
        iter = 0
        etem2 = []
        while error_des > th and iter < maxiter2:
            error_des = 0
            gradient2 = np.zeros((brain_par, brain_par))
            gradient = np.zeros((brain_par, brain_par))

            for jj in range(skip_bold, t - 1):
                error_des += np.linalg.norm(
                    np.dot(M, averaged_bold_seq[jj, :])
                    + np.dot(M_2nstep, averaged_bold_seq[jj, :])
                    - averaged_bold_seq[jj + 1, :]
                )

            error_des *= 0.5
            etem2.append(error_des)

            for jj in range(skip_bold, t - 1):
                gradient2 += (
                    np.dot(M, averaged_bold_seq[jj, :])
                    + np.dot(M_2nstep, averaged_bold_seq[jj, :])
                    - averaged_bold_seq[jj + 1, :]
                )

            M_2nstep = M_2nstep - eta2 * gradient2
            M_2nstep = M_2nstep * noconn_M2
            iter += 1

    etem = np.array(etem)
    etem = etem[etem != 0]
    if use_multistep:
        etem2 = np.array(etem2)
        etem2 = etem2[etem2 != 0]

    np.save(f"{connectome_folder}{name_fold}_eff_conn1.npy", M)

name_session = "path/to/session"
name_fold = "MR_9421819_1328"
norm_opt = 1
use_multistep = 1
use_deconv = 1

structured_G_causality(name_session, name_fold, norm_opt, use_multistep, use_deconv)


FileNotFoundError: No such file or no access: 'path/to/session/4/NIFTI/BOLDrestingCAP_pp_final.nii'